<a href="https://www.kaggle.com/code/prashantshinde98/hyperparametertuning-in-ann-using-kerastuner?scriptVersionId=215529292" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/loan-approval-classification-data/loan_data.csv')
df.head(5)

# Data Analysis

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

### Conclusion:
- data needs to be scaled

In [ ]:
y = df['loan_status']
df= df.drop(columns=['loan_status'])
df.head()

x,y separated

In [ ]:
df.shape

# Preprocessing

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).columns

In [ ]:
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(df[numeric_cols])
scaled_numeric_df = pd.DataFrame(scaled_numeric, columns=numeric_cols)

In [ ]:
encoder = LabelEncoder()
encoded_non_numeric = df[non_numeric_cols].apply(encoder.fit_transform)

In [ ]:
final_df = pd.concat([scaled_numeric_df, encoded_non_numeric], axis=1)
final_df.head()

# Model Building

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(final_df,y,test_size=0.2,random_state=4)

In [ ]:
x_train.head(5)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

### import libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
import kerastuner as kt

## Hyperparameter Tuning

In [ ]:
def build_model(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1, max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                           activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu']),
                           input_dim=13))
            model.add(Dropout(hp.Choice('dropout'+str(i), values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                           activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu'])))
            model.add(Dropout(hp.Choice('dropout'+str(i), values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter +=1
    model.add(Dense(1,activation='sigmoid')) 
    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd']),
                                      loss='binary_crossentropy', metrics=['accuracy'])
    return model
           

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
tuner = kt.RandomSearch(hypermodel=build_model, objective='val_accuracy', max_trials=5, directory='mydir')
tuner.search(x_train,y_train, epochs=50, validation_split=0.2)

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model_final = tuner.get_best_models(num_models=1)[0]

In [ ]:
model_final.fit(x_train, y_train, epochs=200, initial_epoch=50,validation_split=0.2, callbacks=callback)

In [ ]:
model_final.summary()

In [ ]:
#y_pred = model_final.predict(x_test)

# Testing on unseen data

In [ ]:
loss, accuracy = model_final.evaluate(x_test, y_test, verbose=0)
print(accuracy)

In [ ]:
print(loss)

### conclusion
- accuracy on unseen data is 0.9134